In [1]:
import spacy
import numpy as np
import pandas as pd
import os, sys, time

cwd = os.getcwd()
frameworkDir = os.path.abspath(os.path.join(cwd, os.pardir, 'src'))
sys.path.append(frameworkDir)

from dackar.pipelines.CustomPipelineComponents import aliasResolver
from dackar.utils.nlp.nlp_utils import resetPipeline

nlp = spacy.load("en_core_web_lg")
# import logging
# logger = logging.getLogger(__name__)

# ch = logging.StreamHandler()
# logger.addHandler(ch)

ERROR: Unable to import spellchecker No module named 'indexer'
Please try to install it via: 'pip install spellchecker'


Warming up PyWSD (takes ~10 secs)... took 2.4493210315704346 secs.


In [2]:
# aliasResolver pipeline should always after "entity_ruler"
pipelines = ['aliasResolver']
resetPipeline(nlp, pipelines)
print(nlp.pipeline)

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x346dd7a10>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x346dd7110>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x375919150>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x375a89d90>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x375a63a90>), ('aliasResolver', <function aliasResolver at 0x16b0fede0>)]


In [3]:
text="1-91120-P1, CLEAN PUMP AND MOTOR. 1-91120-PM1 REQUIRES OIL. 91120, CLEAN TRASH SCREEN"

In [4]:
# Load module from DACKAR
from dackar.pipelines.GeneralEntity import GeneralEntity
from spacy import displacy
# Load pattern generation from DACKAR
from dackar.utils.nlp.nlp_utils import generatePatternList

pipeline = [pipe for (pipe,_) in nlp.pipeline]
if "enity_ruler" in pipeline:
    nlp.remove_pipe("entity_ruler")
if "ner" in pipeline:
    nlp.remove_pipe("ner")

# Specify Entities Labels and IDs
entLabel = "cws_component"        # user defined entity label
entId = "OPM"  
entIDList = ['1-91120-P1', '1-91120-PM1', '91120']
patternsEnts = generatePatternList(entIDList, label=entLabel,    id=entId,    nlp=nlp, attr="LEMMA")

generalEntity = GeneralEntity(nlp, patternsEnts)

doc = nlp(text)
displacy.render(doc, style='ent', jupyter=True)

In [5]:
for ent in doc.ents:
    print(ent.text, ent._.alias)

1-91120-P1 unit 1 pump
1-91120-PM1 unit 1 pump motor
91120 pump
